In [1]:
pip install torch transformers langchain_pinecone datasets pinecone-client langchain-community scikit-learn langchain datasets transformers==4.17

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install wandb --upgrade

     ---------------------------------------- 0.0/67.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/67.9 kB ? eta -:--:--
     ----------------- -------------------- 30.7/67.9 kB 262.6 kB/s eta 0:00:01
     ---------------------------------- --- 61.4/67.9 kB 469.7 kB/s eta 0:00:01
     -------------------------------------- 67.9/67.9 kB 369.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.8 MB 5.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 20.0 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 24.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/897.5 kB ? eta -:--:--
   --------------------------------------- 897.5/897.5 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.19.2 requires transformers[sentencepiece]<4.41.0,>=4.26.0, but you have transformers 4.17.0 which is incompatible.
sentence-transformers 2.6.1 requires transformers<5.0.0,>=4.32.0, but you have transformers 4.17.0 which is incompatible.


In [9]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install accelerate -U


  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
   ---------------------------------------- 0.0/302.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/302.6 kB ? eta -:--:--
   -- ------------------------------------ 20.5/302.6 kB 165.2 kB/s eta 0:00:02
   ----- --------------------------------- 41.0/302.6 kB 281.8 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/302.6 kB 774.0 kB/s eta 0:00:01
   ---------------------------------------- 302.6/302.6 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.2
    Uninstalling accelerate-0.29.2:
      Successfully uninstalled accelerate-0.29.2


# Setup

In [ ]:
# Python version
import sys 
print(sys.version)

3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]


In [ ]:
# Environment Variables
from dotenv import load_dotenv
import yaml
import os
import tiktoken

# Load env
load_dotenv()

False


We need the following torch configuration:

In [ ]:
# Torch config
from torch import cuda, bfloat16, float16
import torch

# Torch options
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Parameters

In [ ]:
# Load parameters from YAML file
import os

# Change the current working directory to the directory containing the YAML file
os.chdir('/notebooks/TFM_LAW_LLM')

# Load parameters from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Use optimum
use_optimum = config["use_optimum"]

# Show
use_optimum

False

# References

# Directory

In [ ]:
# Set directory to file location
from pathlib import Path
notebook_location = Path(os.path.abspath(''))
os.chdir(notebook_location)

# Get the current working directory
current_directory = os.getcwd()
current_directory

'C:\\Users\\polri\\Desktop\\Git_TFM\\TFM_LAW_LLM'

# Libraries

In [10]:
# General utility libraries
import os 

# PyTorch library for deep learning
import torch  

# HuggingFace Transformers library for natural language processing
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer 

# Dataset handling library from HuggingFace
from datasets import Dataset  

# Pinecone related
import pinecone  
from pinecone import Pinecone  

# LangChain integration with Pinecone
from langchain_pinecone import PineconeVectorStore  

# Transformers pipeline 
from transformers import pipeline  

# LangChain library for creating embeddings with HuggingFace
from langchain.embeddings.huggingface import HuggingFaceEmbeddings  

# Scikit-learn library for machine learning
from sklearn.model_selection import train_test_split  


# Pinecone

In [11]:
# Initialize Pinecone
api_key = os.getenv('PINECONE_API_KEY')
index_name = config["index_name"]
pinecone = Pinecone(api_key = "api_key")

# Connect
index = pinecone.Index(index_name)

# Index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1068}},
 'total_vector_count': 1068}

## Embeddings

In [ ]:
# Load the embedding model
embed_model_id = config["embed_model_id"]
embed_model = HuggingFaceEmbeddings(model_name=embed_model_id, model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'})

c:\Users\polri\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
401 Client Error. (Request ID: Root=1-665df042-64c49e2e19465a9c026af3a0)

Repository Not Found for url: https://huggingface.co/dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn/resolve/main/model.safetensors.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
User Access Token "nacho" is expired


OSError: dariolopez/roberta-base-bne-finetuned-msmarco-qa-es-mnrl-mn is not a local folder or a valid repository name on 'https://hf.co'.

## Data

In [ ]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation="only_second",  # Truncate the context, not the question
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )

    offset_mapping = inputs.pop("offset_mapping").tolist()
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        start_char = examples["answers"][i]["answer_start"]
        end_char = start_char + len(examples["answers"][i]["text"])

        # Find the start and end token indices that correspond to the start and end character positions
        start_token_idx = None
        end_token_idx = None

        for j, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token_idx = j
            if start < end_char <= end:
                end_token_idx = j
            if start_token_idx is not None and end_token_idx is not None:
                break

        if start_token_idx is None or end_token_idx is None:
            start_token_idx = 0
            end_token_idx = 0

        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

Creation of a synthetic dataset with, questions, answers and contexts to train the model.

In [ ]:
data = {
    "question": [
        "¿Cuál es la ley que regula el derecho al honor y la intimidad?",
        "¿Qué artículo de la Constitución Española establece la igualdad ante la ley?",
        "¿Qué establece el artículo 15 de la Constitución Española?",
        "¿Qué es el habeas corpus según la ley española?",
        "¿Qué regula la Ley Orgánica 3/2007?",
        "¿Qué establece el artículo 47 de la Constitución Española?",
        "¿Cuál es el objetivo de la Ley de Enjuiciamiento Criminal?",
        "¿Qué regula el artículo 20 de la Constitución Española?",
        "¿Qué es el Tribunal Constitucional en España?",
        "¿Cuál es el propósito de la Ley de Propiedad Intelectual?",
        "¿Qué establece el artículo 18 de la Constitución Española?",
        "¿Qué es la Ley de Transparencia?",
        "¿Qué regula el artículo 21 de la Constitución Española?",
        "¿Qué es el derecho de asilo según la ley española?",
        "¿Qué establece la Ley de Protección de Datos Personales?",
        "¿Qué es el recurso de amparo según la Constitución Española?",
        "¿Qué establece el artículo 27 de la Constitución Española?",
        "¿Qué regula la Ley General de la Seguridad Social?",
        "¿Qué es el Tribunal Supremo en España?",
        "¿Qué regula la Ley de Arrendamientos Urbanos?"
    ],
    "context": [
        "La Ley Orgánica 1/1982 regula el derecho al honor, a la intimidad personal y familiar y a la propia imagen.",
        "El artículo 14 de la Constitución Española establece que los españoles son iguales ante la ley.",
        "El artículo 15 de la Constitución Española establece el derecho a la vida y a la integridad física y moral.",
        "El habeas corpus es un procedimiento que protege la libertad individual contra arrestos y detenciones arbitrarias.",
        "La Ley Orgánica 3/2007 regula la igualdad efectiva entre mujeres y hombres.",
        "El artículo 47 de la Constitución Española establece el derecho a disfrutar de una vivienda digna y adecuada.",
        "La Ley de Enjuiciamiento Criminal regula el proceso penal en España.",
        "El artículo 20 de la Constitución Española regula la libertad de expresión y de información.",
        "El Tribunal Constitucional es el intérprete supremo de la Constitución Española.",
        "La Ley de Propiedad Intelectual protege los derechos de los autores sobre sus obras.",
        "El artículo 18 de la Constitución Española regula el derecho a la intimidad personal y familiar y a la inviolabilidad del domicilio.",
        "La Ley de Transparencia promueve la transparencia de la actividad pública y el derecho de acceso a la información.",
        "El artículo 21 de la Constitución Española regula el derecho de reunión pacífica y sin armas.",
        "El derecho de asilo protege a las personas que buscan refugio en España por persecución en sus países de origen.",
        "La Ley de Protección de Datos Personales regula el tratamiento de datos personales y la protección de los derechos de los individuos.",
        "El recurso de amparo es un mecanismo constitucional que protege los derechos fundamentales.",
        "El artículo 27 de la Constitución Española establece el derecho a la educación.",
        "La Ley General de la Seguridad Social regula la protección social en España.",
        "El Tribunal Supremo es el órgano jurisdiccional superior en todos los órdenes en España.",
        "La Ley de Arrendamientos Urbanos regula los contratos de alquiler de viviendas y locales."
    ],
    "answers": [
        {"answer_start": 0, "text": "Ley Orgánica 1/1982"},
        {"answer_start": 0, "text": "artículo 14"},
        {"answer_start": 0, "text": "artículo 15"},
        {"answer_start": 0, "text": "habeas corpus"},
        {"answer_start": 0, "text": "Ley Orgánica 3/2007"},
        {"answer_start": 0, "text": "artículo 47"},
        {"answer_start": 0, "text": "Ley de Enjuiciamiento Criminal"},
        {"answer_start": 0, "text": "artículo 20"},
        {"answer_start": 0, "text": "Tribunal Constitucional"},
        {"answer_start": 0, "text": "Ley de Propiedad Intelectual"},
        {"answer_start": 0, "text": "artículo 18"},
        {"answer_start": 0, "text": "Ley de Transparencia"},
        {"answer_start": 0, "text": "artículo 21"},
        {"answer_start": 0, "text": "derecho de asilo"},
        {"answer_start": 0, "text": "Ley de Protección de Datos Personales"},
        {"answer_start": 0, "text": "recurso de amparo"},
        {"answer_start": 0, "text": "artículo 27"},
        {"answer_start": 0, "text": "Ley General de la Seguridad Social"},
        {"answer_start": 0, "text": "Tribunal Supremo"},
        {"answer_start": 0, "text": "Ley de Arrendamientos Urbanos"}
    ]
}

# Create a Dataset object
dataset = Dataset.from_dict(data)

# Train-Test Split for model Training

In [ ]:
# Split data into train and evaluation sets
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2).values()

# Apply the preprocessing function
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Fine-Tuning Model

In [ ]:
# Let's define which model we will use
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/logs", 
    logging_steps=100,     
)

We now train the model.

In [ ]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)

# Train the model
trainer.train()

model.save_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")
tokenizer.save_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")

In [ ]:
# Load the tokenizer and the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("/notebooks/TFM_LAW_LLM/fine_tuning_trained_model/results")  

# Question and Context

In [ ]:
question = "¿Cual es la diferencia entre un homicidio y un asesinato?"
context = """
En el derecho penal español, tanto el homicidio como el asesinato son delitos que implican la muerte de una persona, pero se diferencian en sus características y gravedad. 
El homicidio, regulado en el artículo 138 del Código Penal, se define como causar la muerte a otra persona sin que concurran circunstancias agravantes. 
Por otro lado, el asesinato, tipificado en el artículo 139 del Código Penal, es una forma agravada de homicidio que incluye circunstancias específicas como la alevosía, el ensañamiento o la comisión del delito para facilitar otro delito o evitar ser descubierto.
Estas diferencias se reflejan en las penas aplicables: el homicidio conlleva una pena de prisión de 10 a 15 años, mientras que el asesinato se castiga con una pena de 15 a 25 años, pudiendo incluso llegar a la prisión permanente revisable en casos extremadamente graves.
"""

# Input Pipeline

In [ ]:
# Tokenize the input
inputs = tokenizer(
    question,
    context,
    max_length=512,
    truncation="only_second",
    padding="max_length",
    return_tensors="pt"
)

# Get the input IDs and attention mask
input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

# Output of the Fine-Tuned model

In [ ]:
# Run the model
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Get the start and end positions
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Convert the scores to probabilities
start_prob = torch.softmax(start_scores, dim=1)
end_prob = torch.softmax(end_scores, dim=1)

# Get the most probable start and end positions
start_index = torch.argmax(start_prob)
end_index = torch.argmax(end_prob)

# Extract the answer from the context
all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
answer_tokens = all_tokens[start_index:end_index + 1]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f"Question: {question}")
print(f"Answer: {answer}")